### 刪除所有資源並跳過受保護的角色

In [2]:
import boto3
from botocore.exceptions import ClientError


def delete_all_resources():
    # 建立 IAM 客戶端
    iam = boto3.client("iam")
    # 建立 S3 客戶端
    s3 = boto3.client("s3")
    # 建立 Elastic Beanstalk 客戶端
    eb = boto3.client("elasticbeanstalk")
    # 建立 EC2 客戶端
    ec2 = boto3.client("ec2")
    # 建立 CloudFormation 客戶端
    cloudformation = boto3.client("cloudformation")
    # 建立 RDS 客戶端
    rds = boto3.client("rds")

    # 列出所有 IAM 角色
    roles = iam.list_roles()["Roles"]
    for role in roles:
        # 獲取角色名稱
        role_name = role["RoleName"]
        # 如果角色名稱中不包含 "AWSServiceRole"
        if "AWSServiceRole" not in role_name:
            try:
                # 刪除角色前先分離所有附加的政策
                attached_policies = iam.list_attached_role_policies(
                    RoleName=role_name)[
                    "AttachedPolicies"
                ]
                for policy in attached_policies:
                    iam.detach_role_policy(
                        RoleName=role_name,
                        PolicyArn=policy["PolicyArn"]
                    )

                # 刪除角色
                iam.delete_role(RoleName=role_name)
                # 打印已刪除的角色名稱
                print(f"刪除角色：{role_name}")
            except ClientError as e:
                # 打印錯誤信息
                print(f"無法刪除角色：{role_name}: {e}")

    # 列出所有 S3 Bucket
    buckets = s3.list_buckets()["Buckets"]
    for bucket in buckets:
        # 獲取Bucket名稱
        bucket_name = bucket["Name"]
        try:
            # 刪除Bucket前先刪除所有對象
            objects = s3.list_objects_v2(Bucket=bucket_name).get("Contents", [])
            for obj in objects:
                s3.delete_object(Bucket=bucket_name, Key=obj["Key"])
            # 刪除Bucket
            s3.delete_bucket(Bucket=bucket_name)
            # 打印已刪除的Bucket名稱
            print(f"已刪除的Bucket ：{bucket_name}")
        except ClientError as e:
            # 打印錯誤信息
            print(f"無法刪除Bucket ：{bucket_name}: {e}")

    # 列出所有 Elastic Beanstalk 應用
    applications = eb.describe_applications()["Applications"]
    for app in applications:
        # 獲取應用名稱
        app_name = app["ApplicationName"]
        try:
            # 刪除應用
            eb.delete_application(ApplicationName=app_name)
            # 打印已刪除的應用名稱
            print(f"已刪除的 EB 應用程式：{app_name}")
        except ClientError as e:
            # 打印錯誤信息
            print(f"無法刪除 EB 應用程式 {app_name}: {e}")

    # 列出所有 EC2 實例
    instances = ec2.describe_instances()["Reservations"]
    for reservation in instances:
        for instance in reservation["Instances"]:
            # 獲取實例 ID
            instance_id = instance["InstanceId"]
            try:
                # 終止實例
                ec2.terminate_instances(InstanceIds=[instance_id])
                # 打印已終止的實例 ID
                print(f"T終止的 EC2 執行個體：{instance_id}")
            except ClientError as e:
                # 打印錯誤信息
                print(f"無法終止 EC2 實例{instance_id}: {e}")

    # 列出所有 CloudFormation 堆疊
    stacks = cloudformation.describe_stacks()["Stacks"]
    for stack in stacks:
        try:
            # 刪除堆疊
            cloudformation.delete_stack(StackName=stack["StackName"])
            # 打印已刪除的堆疊名稱
            print(f"刪除的 CloudFormation 堆疊：{stack['StackName']}")
        except ClientError as e:
            print(
                f"無法刪除 CloudFormation 堆疊：{stack['StackName']}: {e}"
            )  # 打印錯誤信息

    # 列出所有 RDS 實例
    instances = rds.describe_db_instances()["DBInstances"]
    for instance in instances:
        # 獲取實例 ID
        instance_id = instance["DBInstanceIdentifier"]
        try:
            rds.delete_db_instance(
                DBInstanceIdentifier=instance_id,
                SkipFinalSnapshot=True,
                DeleteAutomatedBackups=True,
            )
            # 打印已刪除的 RDS 實例 ID
            print(f"刪除的RDS實例：{instance_id}")
        except ClientError as e:
            # 打印錯誤信息
            print(f"無法刪除 RDS 實例：{instance_id}: {e}")


if __name__ == "__main__":
    # 執行刪除所有資源的函數
    delete_all_resources()